
    SET ANSI_NULLS ON
    GO
    SET QUOTED_IDENTIFIER ON
    GO

==============================================================================================================================================================

    Name: Yemisi Adeoluwa
    Date: 07/11/19
    Description: A daily extract for  New Services, New PAs, New Scheduled shifts without 1:2 Service Type and Scheduled Shifts for 1:2 service type for
    Non leave, Waking Nights, Sleepovers, Non / Personal Care shifts happening in the future in North Ayrshire (_NAC)
    Comment: Subshift that has been published

==============================================================================================================================================================

    ALTER PROC [dbo].[DM_CM2000_Ayrshire]
    AS

In [ ]:
DECLARE  @OrgName Nvarchar(100) = 'A***'
        ,@ShiftName Nvarchar(100) = 'N*** P** C***'
		,@ShiftName1 Nvarchar(100) = 'W* N**'
		,@ShiftName2 Nvarchar(100) = 'S*****'
		,@ShiftName3 Nvarchar(100) = 'P*** C****'
		,@ShiftName4 Nvarchar(100) = 'S*** W*** N***'
		,@ShiftName5 Nvarchar(100) = 'S*** S****'
		,@ShiftName6 Nvarchar(100) = 'N** L***'
		,@PublishDate Nvarchar(15) = 'SS.P**_Date'

BEGIN

     Query 1: New Services

In [ ]:
SELECT DISTINCT

         CASE WHEN CAST(CAST(SP1.Start_Date_SK AS VARCHAR(255)) as date) >= (GETDATE()-41) THEN '7'
              ELSE '' 
               END [Data Type ID]

	    ,'' [Unique Visit ID]

		,'' [Date]
		,'' [Start Time]
        ,''+ ',' [End Time] 
		,SP1.Shift_Property_Code + ',,,' [Unique Client ID]	
        ,LEFT(SP1.SHIFT_PROPERTY_DESCRIPTION,CHARINDEX(' ',SP1.SHIFT_PROPERTY_DESCRIPTION + ' ') -2) [Client Forename]
	    ,RIGHT(SP1.SHIFT_PROPERTY_DESCRIPTION,  6) [Client Surname] 
		,'' +',,,,' [Unique Carer ID]
		,'' [Carer Forename]
		,'' +',,,,,,,,,'[Carer Surname]

		,'3024926' [Social Services ID] -- TOM PLEASE CAN YOU KINDLY DELETE THIS ONE AND USE THE BELOW ONE THANK YOU
		--,SPR.RATE [Social Services ID]   --TOM PLEASE CAN YOU KINDLY USE THIS INSTEAD OF THE ABOVE ONE AFTER ITS TABLE DM_DIM_SHIFT_PROPERTY_REVISION HAS BEEN ADDED
		
		,CONVERT(CHAR(10),SP1.Start_Date_SK,103) + ',,,,,' [Start Date]
		,''+ ',,,,,,,,,' [Contact_Type_ID]

FROM   DM_Fact_SCH**_SUB** SS  LEFT JOIN DM_DIM_SCH**_S*** S
       ON S.Sch***_Sh**_ID = SS.Sch**_Sh***_ID LEFT JOIN DM_DIM_EMP*** EE 
       *******************************

WHERE    EE.LOGIN_ID IS NOT NULL
         AND CAST(CAST(SP1.Start_Date_SK AS VARCHAR(255)) as date) >= (GETDATE()-41)
		 AND SS.Start_Date >= (GETDATE()-7) 
		 AND @PublishDate IS NOT NULL 
	     AND ORGPA.ORG_NAME = @OrgName
	     AND SS.SHIFT_TYPE_NAME IN(@ShiftName,@ShiftName1,@ShiftName2,@ShiftName3,@ShiftName4,@ShiftName5)
		 AND SP1.Shift_Property_Code LIKE ('%N**%')

UNION

    Query 2: New PAs

In [ ]:
SELECT DISTINCT

           CASE WHEN CAST(CAST(EE.Start_Date_SK AS VARCHAR(255)) as date) >= (GETDATE()-41) THEN ' 11 '
		        ELSE ''
				    END [Data Type ID]

		,'' [Unique Visit ID]
		,'' [Date]
		,'' [Start Time]
        ,''+ ',' [End Time] 
		,''+ ',,,' [Unique Client ID]	
		,'' [Client Forename]
	    ,'' [Client Surname]
		,EE.LOGIN_ID +',,,,' [Unique Carer ID]
		,EE.FIRST_NAME [Carer Forename]
		,EE.LAST_NAME +',,,,,,,,,'[Carer Surname]
		,'' [Social_Services_ID]
		,CONVERT(CHAR(10),EE.Start_Date_SK,103) + ',,,,,' [Start Date]
		,''+ ',,,,,,,,,' [Contact_Type_ID]

FROM   DM_Fact_S**_S** SS  LEFT JOIN DM_DIM_S**_SHIFT S
       ON S.S**_Shift_ID = SS.Sche**ID ****************
                
WHERE    EE.LOGIN_ID IS NOT NULL
         AND CAST(CAST(EE.Start_Date_SK AS VARCHAR(255)) as date) >= (GETDATE()-41)
		 AND SS.Start_Date >= (GETDATE()-7) 
		 AND @PublishDate IS NOT NULL 
	     AND ORGPA.ORG_NAME = @OrgName
	     AND SS.SHIFT_TYPE_NAME IN(@ShiftName,@ShiftName1,@ShiftName2,@ShiftName3,@ShiftName4,@ShiftName5)
		 AND SP1.Shift_Property_Code LIKE ('%N**%')   

 UNION

     Query3:  New scheduled shifts without 1:2 service type

In [ ]:
SELECT DISTINCT
         CASE WHEN SP1.Shift_Property_Code LIKE '%N**%' THEN '3'
              ELSE '' 
			     END [Data Type ID]

		,CAST(CAST(SS.Schedule_Subshift_ID AS VARCHAR(20)) + '  ' AS VARCHAR(24)) [Unique Visit ID]
		,CONVERT(CHAR(10),SS.START_DATE, 103) [Date]
		,CONVERT(VARCHAR(5),SS.START_DATE,108) [Start Time]
        ,CONVERT(VARCHAR(5),SS.END_DATE,108) + ',' [End Time] 
		,SP1.Shift_Property_Code + ',,,' [Unique Client ID]	
		,'' [Client Forename]
	    ,'' [Client Surname]  
		,EE.LOGIN_ID +',,,,' [Unique Carer ID]
		,'' [Carer Forename]
		,'' +',,,,,,,,,'[Carer Surname]
		,'' [Social_Services_ID]
        ,''+ ',,,,,' [Start Date]
		,SS.SHIFT_TYPE_NAME + ',,,,,,,,,' [Contact_Type_ID]

FROM   DM_Fact_S**  SS  LEFT JOIN DM_DIM_S** S
       ON S.S**ID = SS.Sch**_S**_ID LEFT JOIN DM_DIM_EM** EE 

                
WHERE  @PublishDate IS NOT NULL  
        AND EE.LOGIN_ID IS NOT NULL
	    AND ORGPA.ORG_NAME = @OrgName
	    AND SS.SHIFT_TYPE_NAME IN(@ShiftName,@ShiftName1,@ShiftName2,@ShiftName3,@ShiftName4,@ShiftName5)
		AND SP1.Shift_Property_Code LIKE ('%N**%')
		AND SP2.SHIFT_PROPERTY_CODE != '**Su**t'
	    AND SS.Start_Date >= (GETDATE()-7) 

UNION

    Query 4: schedule shifts for 1:2 service type, that shows the second service with the same Unique Visit ID has 2 at the end

In [ ]:
SELECT DISTINCT

       CASE WHEN SP2.SHIFT_PROPERTY_CODE = '1:2 Support' THEN '3'
             ELSE '' 
			    END [Data Type ID]

       ,CASE Row_Number() OVER (Partition By CAST(SS.Schedule_Subshift_ID AS VARCHAR(20)) Order By SP1.Shift_Property_Code) 
              WHEN 1 then CAST(SS.Schedule_Subshift_ID AS VARCHAR(20))
              WHEN 2 then CAST(CAST(SS.Schedule_Subshift_ID AS VARCHAR(20)) + '2' AS VARCHAR(24)) 
                   END [Unique Visit ID]

		,CONVERT(CHAR(10),SS.START_DATE, 103) [Date]
		,CONVERT(VARCHAR(5),SS.START_DATE,108) [Start Time]
        ,CONVERT(VARCHAR(5),SS.END_DATE,108) + ',' [End Time] 
		,SP1.Shift_Property_Code + ',,,' [Unique Client ID]	
		,'' [Client Forename]
	    ,'' [Client Surname] 
		,EE.LOGIN_ID +',,,,' [Unique Carer ID]
		,'' [Carer Forename]
		,'' +',,,,,,,,,'[Carer Surname]
		,'' [Social_Services_ID]
        ,''+ ',,,,,' [Start Date]
		,SS.SHIFT_TYPE_NAME + ',,,,,,,,,' [Contact_Type_ID]
		

FROM   DM_Fact_SS SS  LEFT JOIN DM_DIM_SS S
       ON S.Sc_S_ID = SS.S_S_ID LEFT JOIN DM_DIM_EMP EE 
                    
WHERE   @PublishDate IS NOT NULL 
        and SP2.SHIFT_PROPERTY_CODE = '1:2 S**'
        AND EE.LOGIN_ID IS NOT NULL
	    AND ORGPA.ORG_NAME = @OrgName
	    AND SS.SHIFT_TYPE_NAME IN(@ShiftName,@ShiftName1,@ShiftName2,@ShiftName3,@ShiftName4,@ShiftName5)
		AND SP1.Shift_Property_Code LIKE ('%N**%')
	    AND SS.Start_Date >= (GETDATE()-7)  
		
ORDER BY [Client Forename] DESC

END